In [ ]:
import pandas as pd
from pathlib import Path

from se_coast_strandings import make_dt_col, make_cyclic, fetch_weather_context

In [ ]:
DATA_DIR = Path("../data/raw")

In [ ]:
df = pd.read_excel(DATA_DIR / "UNC-DataRequest-01302026.xlsx", sheet_name="2015-2024")

In [ ]:
df["mms_observation_dt"] = make_dt_col(
    df["Day of Observation"], df["Month of Observation"], df["Year of Observation"]
)

In [ ]:
df = df[df["mms_observation_dt"] >= pd.Timestamp("2016-01-02")]

In [ ]:
df["mms_observation_dt"].min()

In [ ]:
df["dayofweek_sin"], df["dayofweek_cos"] = make_cyclic(
    df["mms_observation_dt"].dt.dayofweek, 7, name="dayofweek"
)
df["month_sin"], df["month_cos"] = make_cyclic(
    df["mms_observation_dt"].dt.month, 12, name="month"
)

In [ ]:
df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")

In [ ]:
df.drop(index=df[df["Latitude"].isna() | df["Longitude"].isna()].index, inplace=True)
df[["Latitude", "Longitude"]].isna().sum()

In [ ]:
df = df.reset_index(drop=True)
df.head(10)

In [ ]:
(
    df["mms_observation_dt"].sort_values().iloc[1000:1010].max(),
    df["mms_observation_dt"].sort_values().iloc[1000:1010].min(),
)


In [ ]:
test_df = df.sort_values("mms_observation_dt").iloc[1000:1010].reset_index(drop=True)
test_df

In [ ]:
weather = fetch_weather_context(
    test_df,
    lat_column="Latitude",
    lon_column="Longitude",
    date_column="mms_observation_dt",
    daily_variables=["temperature_2m_max", "temperature_2m_min"],
    include_deltas=True,
)


In [ ]:
weather

In [ ]:
test_df = pd.concat([test_df, weather], axis=1)
test_df